<a href="https://colab.research.google.com/github/96jonesa/CSE-517-Project/blob/main/scaffolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

#GRU

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size, hidden_size)

    def forward(self, input, h_0):
        output, hn = self.gru(input, h_0)
        return output, hn

#Self-Attention

In [ ]:
# attention weights are softmax(left^T W right) where W is learned parameter matrix

class TemporalAttention(nn.Module):
    def __init__(self, left_size, right_size, weights_size):
        super(TemporalAttention, self).__init__()
        self.left_size = left_size
        self.right_size = right_size
        self.weights_size = weights_size

        self.bilinear = nn.Bilinear(self.left_size, self.right_size, weights_size, bias=False)
        self.softmax = nn.Softmax()

    def forward(self, left, right):
        attention_weights = self.softmax(self.bilinear(left, right))
        #output_features = torch.mm(attention_weights, left)

        return attention_weights#, output_features

#Blending

In [ ]:
# output is ReLU(left^T W right + b) where W is a learned paramater matrix
# and b is a learned bias

class Blend(nn.Module):
    def __init__(self, left_size, right_size, output_size):
        super(Blend, self).__init__()
        self.left_size = left_size
        self.right_size = right_size
        self.output_size = output_size

        self.bilinear = nn.Bilinear(self.left_size, self.right_size, output_size, bias=True)
        self.relu = nn.ReLU()
    
    def forward(self, left, right):
        output = self.relu(self.bilinear(left, right))

        return output

#Single-Headed Graph Attention Network (SGAT)

In [ ]:
# need shared learned parameter matrix W to multiply against each input vector

class SharedLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(SharedLinear, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.linear = nn.Linear(input_size, output_size, bias=False)
    
    def forward(self, input):
        output = self.linear(input)

        return output

In [ ]:
# merge code with MGAT code to form general case GAT code

class SGAT(nn.Module):
    def __init__(self, input_size, weights_size, leakyrelu_slope):
        super(SGAT, self).__init__()
        self.input_size = input_size
        self.weights_size = weights_size
        self.leakyrelu_slope = leakyrelu_slope
        
        self.linear = nn.Linear(2 * input_size, weights_size, bias=False)
        self.leakyrelu = nn.LeakyReLU(self.leakyrelu_slope)
        self.softmax = nn.Softmax()

    def forward(self, input, neighborhood):
        attention_weights = self.softmax(self.leakyrelu(self.linear(torch.cat(input, neighborhood))))  # check concatenation axis

        return attention_weights

#Multi-Headed Graph Attention Network (MGAT)

In [ ]:
# implement this

#Scaffolding

In [ ]:
# implement this